In [27]:
import csv
from getpass import getpass
from time import sleep
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver import Chrome
import nest_asyncio
nest_asyncio.apply()

from selenium import webdriver
# op =webdriver.ChromeOptions()
# op.add_argument('headless')
driver = webdriver.Chrome('./chromedriver')#options=op
driver.get('https://bitcoinmagazine.com/articles')

In [28]:
cards = driver.find_elements_by_xpath('//phoenix-card')
cards

[<selenium.webdriver.remote.webelement.WebElement (session="27238407db67a2fa26014abaf06afb35", element="9f65398e-adb0-4819-9d62-d6a2b3bdfe9c")>,
 <selenium.webdriver.remote.webelement.WebElement (session="27238407db67a2fa26014abaf06afb35", element="5a59c413-2ae5-4faf-837b-7b01d50cedd1")>,
 <selenium.webdriver.remote.webelement.WebElement (session="27238407db67a2fa26014abaf06afb35", element="9cfed79a-0e4e-45e8-8d5b-2d0ffb550467")>,
 <selenium.webdriver.remote.webelement.WebElement (session="27238407db67a2fa26014abaf06afb35", element="6adc4814-fd87-4568-85fc-dff4343381d2")>,
 <selenium.webdriver.remote.webelement.WebElement (session="27238407db67a2fa26014abaf06afb35", element="9fe89d87-283a-476a-8704-23d9fe10ea50")>,
 <selenium.webdriver.remote.webelement.WebElement (session="27238407db67a2fa26014abaf06afb35", element="6597071f-3ef1-4aaa-8d5d-43ea41f52d8d")>,
 <selenium.webdriver.remote.webelement.WebElement (session="27238407db67a2fa26014abaf06afb35", element="7a9ff80a-c827-4574-aafc-d1

In [21]:
#textual title
card = cards[0]
card.find_element_by_xpath('.//h2').text

'HOW PROOF-OF-WORK IS USEFUL BEYOND BITCOIN'

In [25]:
#categories
card.find_element_by_xpath('./div[2]/div//a').text

'CULTURE'

In [26]:
#links
card.find_element_by_xpath('.//a').get_attribute('href')

'https://bitcoinmagazine.com/culture/proof-of-work-useful-beyond-bitcoin'

In [24]:
#published date
url1 = card.find_element_by_xpath('.//a').get_attribute('href') #step1:setup url1 you want to visit in new page
openWebScript = 'window.open("'+url1+'");'
# closeWebScript = "window.open(,_self,);window.close();"
driver.execute_script(openWebScript)
driver.switch_to.window(driver.window_handles[1])
publishedDate = driver.find_element_by_xpath('.//time').get_attribute('datetime')
sleep(2)
driver.close()
driver.switch_to.window(driver.window_handles[0])
publishedDate


# driver.execute_script(closeWebScript)


# driver.execute_script(strScript)
# card.find_element_by_xpath('.//time').get_attribute('datetime')

'2021-12-04T21:00:00-05:00'

In [14]:
import csv
from getpass import getpass
from time import sleep
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver import Chrome
import nest_asyncio
nest_asyncio.apply()

from selenium import webdriver
# op =webdriver.ChromeOptions()
# op.add_argument('headless')
driver = webdriver.Chrome('./chromedriver')#options=op
driver.get('https://bitcoinmagazine.com/articles')

In [15]:
def get_news_data(card):
    """Extract data from news data"""
    if len(driver.window_handles) == 2:
            driver.switch_to.window(driver.window_handles[1])
            driver.close()
            driver.switch_to.window(driver.window_handles[0])
            sleep(1.5)
    try:
        sleep(0.3)
        categories = card.find_element_by_xpath('./div[2]/div//a').text
    except:
        categories = 'None'
        return
    try:
        sleep(0.3)
        title = card.find_element_by_xpath('.//h2').text
    except:
        title ='None'
        return
    try:
        sleep(0.3)
        link = card.find_element_by_xpath('.//a').get_attribute('href')
    except:
        link ='None'
        return
    try:      
        sleep(0.3)
        url1 = card.find_element_by_xpath('.//a').get_attribute('href') #step1:setup url1 you want to visit in new page    
        openWebScript = 'window.open("'+url1+'");'
        driver.execute_script(openWebScript)
        driver.switch_to.window(driver.window_handles[1])
        sleep(3)
        try:
            postdate = driver.find_element_by_xpath('.//time').get_attribute('datetime')
        except:
            postdate = 'None'
            return
        sleep(1)
        driver.close()
        sleep(1)
        driver.switch_to.window(driver.window_handles[0])
    except:
        link = 'None'
        postdate = 'None'
        return
    news_all = (categories, postdate, title, link)
    return news_all

# get all news on the page
data = []
news_ids = set()
last_position = driver.execute_script("return window.pageYOffset;")
scrolling = True

while scrolling:
    page_cards = driver.find_elements_by_xpath('//phoenix-card')
    for card in page_cards[-50:]:
        news = get_news_data(card)
        if news:
            try:
                news_id = ''.join(news)
                if news_id not in news_ids:
                    news_ids.add(news_id)
                    data.append(news)
            except:
                continue
                

    scroll_attempt = 0
    while True:
        try:
            if len(driver.window_handles) == 2:
                driver.switch_to.window(driver.window_handles[1])
                driver.close()
                driver.switch_to.window(driver.window_handles[0])
                sleep(2)
            driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
            sleep(2.6)
            condition = True
            while condition == True:
                try:
                    driver.find_element_by_xpath('//Button[@phx-track-id="load more"]').click()
                    condition = False
                except NoSuchElementException:
                    condition == True

            sleep(5.5)
            crr_position = driver.execute_script("return window.pageYOffset;")
            
            if last_position == crr_position:
                scroll_attempt += 1

                # end of scroll region
                if scroll_attempt >= 10:
                    scrolling = False
                    break
                else:
                    sleep(1.5) #attempt to scroll again
            else:
                last_position = crr_position
                break
        except TimeoutError:
            print("---------------")
            print("程式運行終止！！")
            print("---------------")


TimeoutException: Message: timeout: Timed out receiving message from renderer: 300.000
  (Session info: chrome=96.0.4664.55)


In [16]:
data

[('CULTURE',
  '2021-12-05T16:00:00-05:00',
  'HOW AN 18TH CENTURY GAMBLER PAVED THE WAY FOR CENTRAL BANKS, AND NECESSITATED BITCOIN',
  'https://bitcoinmagazine.com/culture/bitcoin-origins-in-18th-century-france'),
 ('BUSINESS',
  '2021-12-05T13:00:00-05:00',
  'EXAMINING THE LINDY EFFECT AND BITCOIN',
  'https://bitcoinmagazine.com/business/examining-the-lindy-effect-bitcoin'),
 ('CULTURE',
  '2021-12-04T21:00:00-05:00',
  'HOW PROOF-OF-WORK IS USEFUL BEYOND BITCOIN',
  'https://bitcoinmagazine.com/culture/proof-of-work-useful-beyond-bitcoin'),
 ('CULTURE',
  '2021-12-04T17:00:00-05:00',
  'HOW BITCOIN CREATES OUR NEW REALITY',
  'https://bitcoinmagazine.com/culture/how-bitcoin-creates-our-new-reality'),
 ('CULTURE',
  '2021-12-04T16:00:00-05:00',
  'HOW BITCOIN HAS CHARACTERISTICS OF VEBLEN GOODS',
  'https://bitcoinmagazine.com/culture/how-bitcoin-has-characteristics-of-veblen-goods'),
 ('TECHNICAL',
  '2021-12-03T22:00:00-05:00',
  "THE COERCION OF ETHEREUM'S DIFFICULTY BOMB",
  '

In [17]:
import pandas as pd

pd_news_data = pd.DataFrame(data)

In [18]:
pd_news_data

,0,1,2,3
0,CULTURE,2021-12-05T16:00:00-05:00,HOW AN 18TH CENTURY GAMBLER PAVED THE WAY FOR ...,https://bitcoinmagazine.com/culture/bitcoin-or...
1,BUSINESS,2021-12-05T13:00:00-05:00,EXAMINING THE LINDY EFFECT AND BITCOIN,https://bitcoinmagazine.com/business/examining...
2,CULTURE,2021-12-04T21:00:00-05:00,HOW PROOF-OF-WORK IS USEFUL BEYOND BITCOIN,https://bitcoinmagazine.com/culture/proof-of-w...
3,CULTURE,2021-12-04T17:00:00-05:00,HOW BITCOIN CREATES OUR NEW REALITY,https://bitcoinmagazine.com/culture/how-bitcoi...
4,CULTURE,2021-12-04T16:00:00-05:00,HOW BITCOIN HAS CHARACTERISTICS OF VEBLEN GOODS,https://bitcoinmagazine.com/culture/how-bitcoi...
...,...,...,...,...
3028,BUSINESS,2019-05-20T17:19:39-04:00,SEC DELAYS VANECK ETF YET AGAIN,https://bitcoinmagazine.com/business/sec-delay...
3029,BUSINESS,2019-05-20T17:10:44-04:00,‘THE DOLLAR IS GOING TO SELF DESTRUCT’: TALKIN...,https://bitcoinmagazine.com/business/the-dolla...
3030,MARKETS,2019-05-20T14:58:51-04:00,BISQ’S LEAD DEVELOPER MAKES WAY FOR DAO TO TAK...,https://bitcoinmagazine.com/markets/bisqs-lead...
3031,MARKETS,2019-05-20T13:48:21-04:00,"DURING BULLISH APRIL, FIAT-TO-CRYPTO EXCHANGES...",https://bitcoinmagazine.com/markets/during-bul...


In [19]:
pd_news_data.columns = ['Categories', 'Published Date', 'Title', 'Links']
pd_news_data

,Categories,Published Date,Title,Links
0,CULTURE,2021-12-05T16:00:00-05:00,HOW AN 18TH CENTURY GAMBLER PAVED THE WAY FOR ...,https://bitcoinmagazine.com/culture/bitcoin-or...
1,BUSINESS,2021-12-05T13:00:00-05:00,EXAMINING THE LINDY EFFECT AND BITCOIN,https://bitcoinmagazine.com/business/examining...
2,CULTURE,2021-12-04T21:00:00-05:00,HOW PROOF-OF-WORK IS USEFUL BEYOND BITCOIN,https://bitcoinmagazine.com/culture/proof-of-w...
3,CULTURE,2021-12-04T17:00:00-05:00,HOW BITCOIN CREATES OUR NEW REALITY,https://bitcoinmagazine.com/culture/how-bitcoi...
4,CULTURE,2021-12-04T16:00:00-05:00,HOW BITCOIN HAS CHARACTERISTICS OF VEBLEN GOODS,https://bitcoinmagazine.com/culture/how-bitcoi...
...,...,...,...,...
3028,BUSINESS,2019-05-20T17:19:39-04:00,SEC DELAYS VANECK ETF YET AGAIN,https://bitcoinmagazine.com/business/sec-delay...
3029,BUSINESS,2019-05-20T17:10:44-04:00,‘THE DOLLAR IS GOING TO SELF DESTRUCT’: TALKIN...,https://bitcoinmagazine.com/business/the-dolla...
3030,MARKETS,2019-05-20T14:58:51-04:00,BISQ’S LEAD DEVELOPER MAKES WAY FOR DAO TO TAK...,https://bitcoinmagazine.com/markets/bisqs-lead...
3031,MARKETS,2019-05-20T13:48:21-04:00,"DURING BULLISH APRIL, FIAT-TO-CRYPTO EXCHANGES...",https://bitcoinmagazine.com/markets/during-bul...


In [20]:
pd_news_data.to_csv('BitcoinMagazine_data(完整).csv', mode='a', header=True)

In [7]:
driver.find_element_by_xpath('//Button[@phx-track-id="load more"]').click()
sleep(1.5)
crr_position = driver.execute_script("return window.pageYOffset;")

In [8]:
while scrolling:
    page_cards = driver.find_elements_by_xpath('//phoenix-card')
    for card in page_cards[-50:]:
        news = get_news_data(card)
        if news:
            news_id = ''.join(news)
            if news_id not in news_ids:
                news_ids.add(news_id)
                data.append(news)
    
    scroll_attempt = 0
    while True:
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        sleep(3)
        driver.find_element_by_xpath('//Button[@phx-track-id="load more"]').click()
        sleep()
        crr_position = driver.execute_script("return window.pageYOffset;")
        
        if last_position == crr_position:
            scroll_attempt += 1

            # end of scroll region
            if scroll_attempt >= 10:
                scrolling = False
                break
            else:
                sleep(1) #attempt to scroll again
        else:
            last_position = crr_position
            break

StaleElementReferenceException: Message: stale element reference: element is not attached to the page document
  (Session info: chrome=94.0.4606.81)


In [ ]:

#掃全部資料出來, a loop loading trigger
scroll_attempt = 0
while True:
    try:
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        sleep(5)
        crr_position = driver.execute_script("return window.pageYOffset;")
        driver.find_element_by_xpath('//Button[@phx-track-id="load more"]').click()
        sleep(10)
    except:
        break

In [ ]:
#[2].開分頁
#function2:open new page with javascropt command #function: 使用javascript指令在同一個網頁視窗中開啟新分頁
driver.find_element_by_link_text()
print('function:open new page with javascropt command')
url1 = 'http://stylengineer.com/program/' #step1:setup url1 you want to visit in new page
strScript = 'window.open("'+url1+'");'
driver.execute_script(strScript)
print('sleep 5 sec...')
sleep(5)

In [ ]:
for button in buttons: 
    onclick_text = button.get_attribute('onclick') 

In [6]:
data

[('TECHNICAL',
  '2021-10-16T15:00:00-04:00',
  'COULD ADVANCED QUANTUM COMPUTING POSE A RISK TO BITCOIN SECURITY?',
  'https://bitcoinmagazine.com/technical/quantum-computing-and-bitcoin-security'),
 ('MARKETS',
  '2021-10-16T13:51:30-04:00',
  'A GRAYSCALE BITCOIN SPOT ETF COULD BE RIGHT AROUND THE CORNER',
  'https://bitcoinmagazine.com/markets/grayscale-spot-bitcoin-etf-soon'),
 ('CULTURE',
  '2021-10-16T12:00:00-04:00',
  'BITCOIN: THE STATELESS EMERGENCE',
  'https://bitcoinmagazine.com/culture/bitcoin-the-stateless-emergence'),
 ('CULTURE',
  '2021-10-16T00:00:00-04:00',
  'THE BITCOIN RORSCHACH TEST',
  'https://bitcoinmagazine.com/culture/the-bitcoin-rorschach-test'),
 ('MARKETS',
  '2021-10-15T22:00:00-04:00',
  'THE AMERICAN DREAM IS DEAD AND BITCOIN HAS REPLACED IT',
  'https://bitcoinmagazine.com/markets/how-bitcoin-replaces-the-american-dream'),
 ('CULTURE',
  '2021-10-15T20:00:00-04:00',
  'BITCOIN EDUCATION FOR INDONESIA',
  'https://bitcoinmagazine.com/culture/bitcoin-